In [2]:
!pip install torchvision.transformer

ERROR: Could not find a version that satisfies the requirement torchvision.transformer (from versions: none)
ERROR: No matching distribution found for torchvision.transformer


In [3]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

# basic autograd 
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

# build a computational graph
y = w * x + b # y = 2 * x + 3

# compute gradients
y.backward()

# print out the gradients
print(x.grad)
print(w.grad)
print(b.grad)

tensor(2.)
tensor(1.)
tensor(1.)


In [4]:
# create tensors of shape (10, 3) and (10, 2)
x = torch.randn(10, 3)
y = torch.randn(10, 2)

# build a fully connected layer
linear = nn.Linear(3, 2)
print('w: ', linear.weight)
print('b: ', linear.bias)

w:  Parameter containing:
tensor([[ 0.1838,  0.1479,  0.5650],
        [-0.0821, -0.4800,  0.2258]], requires_grad=True)
b:  Parameter containing:
tensor([ 0.5554, -0.2077], requires_grad=True)


In [6]:
# Build loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)
print(criterion, optimizer)


MSELoss() SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [10]:
# Forward pass
pred = linear(x)

# Compute loss.
loss = criterion(pred, y)
print('loss:', loss.item())

# Backward pass.
loss.backward()

# Print out the gradients.
print('dL/dw: ', linear.weight.grad)
print('dL/db: ', linear.bias.grad)

# 1-step gradient descent.
optimizer.step()

# Print out the loss after 1-step gradient descent.
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())

loss: 1.4553918838500977
dL/dw:  tensor([[-0.3979, -0.2021,  1.3191],
        [-0.2496, -0.4713,  1.0693]])
dL/db:  tensor([2.0722, 0.7060])
loss after 1 step optimization:  1.4151173830032349


In [11]:
# Create a numpy array.
x = np.array([[1, 2], [3, 4]])
print(x)
# Convert the numpy array to a torch tensor.
y = torch.from_numpy(x)
print(y)
# Convert the torch tensor to a numpy array.
z = y.numpy()
print(z)


[[1 2]
 [3 4]]
tensor([[1, 2],
        [3, 4]])
[[1 2]
 [3 4]]


In [14]:
# Input pipeline
# Download and construct CIFAR-10 dataset.
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True,
                                             transform=transforms.ToTensor(),
                                             download=True)
# Fetch one data pair (read data from disk).
image, label = train_dataset[0]
print(image.size())
print(label)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ../../data/cifar-10-python.tar.gz to ../../data/
torch.Size([3, 32, 32])
6


In [ ]:
# Data loader (this provides queues and threads in a very simple way).
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64,
                                           shuffle=True)
# When iteration starts, queue and thread start to load data from files.
data_iter = iter(train_loader)

# Mini-batch images and labels.
images, labels = data_iter.next()
print(images,labels)
# Actual usage of the data loader is as below.
for images, labels in train_loader:
    # Training code should be written here.
    
    pass

In [21]:
# Input pipeline for custom dataset
# You should build your custom dataset as below.
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        # Initializer file paths or a list of file names.
        pass
    def __getitem__(self, index):
        # Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        # Preprocess the data (e.g. torchvision.Transform).
        # Return a data pair (e.g. image and label).
        pass
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return 0
# Youc an then use the prebuilt data loader.
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                           batch_size=64,
                                           shuffle=True)


ValueError: ignored

In [22]:
# Pretrained model
# Download and load the pretrained ResNet-18.
resnet = torchvision.models.resnet18(pretrained=True)

# If you want to finetune only the top layer of the model, set as below.
for param in resnet.parameters():
    param.requires_grad = False

# Replace the top layer for finetuning.
resnet.fc = nn.Linear(resnet.fc.in_features, 100) # 100 is an example.

# Forward pass.
images = torch.randn(64, 3, 224, 224)
outputs = resnet(images)
print(outputs.size()) 

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

torch.Size([64, 100])


In [25]:
# Save and load the model
# Save and load the entire model.
torch.save(resnet, 'model.ckpt')
model = torch.load('model.ckpt')

# Save and load only the model parameters (recommended).
torch.save(resnet.state_dict(), 'params.ckpt')
resnet.load_state_dict(torch.load('params.ckpt'))

<All keys matched successfully>